<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import mediapipe as mp
import numpy as np
import cv2
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window
import pandas as pd
from tqdm import tqdm
import gc
from operator import add

2023-08-03 11:43:04.466154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 11:43:09.586737: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [3]:
#cap = cv2.VideoCapture("file://home/kristian/WLASL/videos_organized/address/01018.mp4")
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # face
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
results

mediapipe.python.solution_base.SolutionOutputs

In [5]:
len(results.pose_landmarks.landmark)

33

In [6]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [7]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [8]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [9]:
result_test = extract_keypoints(results)

In [10]:
result_test

array([ 0.54196721,  0.35831186, -1.15823162, ...,  0.        ,
        0.        ,  0.        ])

In [11]:
video_df = spark.read.format("binaryFile").option(
    "pathGlobFilter", "*.mp4").load('file:///home/kristian/WLASL/videos_organized/*').orderBy("path")

In [12]:
video_df

DataFrame[path: string, modificationTime: timestamp, length: bigint, content: binary]

In [13]:
#create an index column for future join
video_df = video_df.withColumn("columnindex",
                               row_number().over(Window().orderBy("path")))

In [14]:
video_df = video_df.drop('length','modification')

In [15]:
video_df2 = video_df.select("path").collect()

In [16]:
#loop to get labels and filename per row
i = 0
target = []
for row in video_df2:
    subdirname = os.path.basename(os.path.dirname(video_df2[i][0]))
    name = os.path.basename(video_df2[i][0]).split('/')[-1]
    target.append([subdirname, name])
    i = i + 1

In [17]:
target

[['a lot', '02123.mp4'],
 ['a lot', '02124.mp4'],
 ['a lot', '02125.mp4'],
 ['a lot', '02126.mp4'],
 ['a lot', '02128.mp4'],
 ['a lot', '02129.mp4'],
 ['a lot', '02130.mp4'],
 ['a lot', '02131.mp4'],
 ['a', '00295.mp4'],
 ['a', '01610.mp4'],
 ['a', '01611.mp4'],
 ['a', '01612.mp4'],
 ['a', '01613.mp4'],
 ['a', '01614.mp4'],
 ['a', '01615.mp4'],
 ['abdomen', '00333.mp4'],
 ['abdomen', '00335.mp4'],
 ['abdomen', '00336.mp4'],
 ['abdomen', '00338.mp4'],
 ['abdomen', '00339.mp4'],
 ['abdomen', '00340.mp4'],
 ['abdomen', '00341.mp4'],
 ['able', '00374.mp4'],
 ['able', '00376.mp4'],
 ['able', '00377.mp4'],
 ['able', '00378.mp4'],
 ['able', '00381.mp4'],
 ['able', '00382.mp4'],
 ['able', '00383.mp4'],
 ['able', '00384.mp4'],
 ['about', '00412.mp4'],
 ['about', '00414.mp4'],
 ['about', '00415.mp4'],
 ['about', '00416.mp4'],
 ['about', '00421.mp4'],
 ['about', '00422.mp4'],
 ['about', '00423.mp4'],
 ['about', '00424.mp4'],
 ['about', '00425.mp4'],
 ['about', '00426.mp4'],
 ['above', '00429.mp4'

In [18]:
targetRDD = sc.parallelize([row[0] for row in target])

In [19]:
labelcount = targetRDD.map(lambda w: (w, 1))

In [20]:
labelcount = labelcount.reduceByKey(add)

In [21]:
labelcount.sortBy(lambda x: x[1],ascending=False).collect()

[('book', 34),
 ('clothes', 23),
 ('drink', 23),
 ('chair', 22),
 ('before', 22),
 ('computer', 22),
 ('like', 19),
 ('go', 19),
 ('thin', 19),
 ('orange', 18),
 ('cousin', 18),
 ('mother', 18),
 ('hot', 18),
 ('yes', 18),
 ('deaf', 18),
 ('candy', 18),
 ('now', 17),
 ('accident', 17),
 ('black', 17),
 ('who', 17),
 ('wrong', 17),
 ('fine', 17),
 ('bowling', 17),
 ('cool', 17),
 ('woman', 17),
 ('no', 17),
 ('pizza', 17),
 ('table', 17),
 ('thanksgiving', 17),
 ('shirt', 17),
 ('bed', 17),
 ('study', 17),
 ('tall', 17),
 ('white', 17),
 ('bird', 16),
 ('color', 16),
 ('family', 16),
 ('finish', 16),
 ('graduate', 16),
 ('forget', 16),
 ('many', 16),
 ('fish', 16),
 ('thursday', 16),
 ('year', 16),
 ('apple', 16),
 ('dog', 16),
 ('africa', 16),
 ('short', 16),
 ('later', 16),
 ('man', 16),
 ('secretary', 16),
 ('change', 16),
 ('help', 16),
 ('visit', 16),
 ('check', 15),
 ('hat', 15),
 ('school', 15),
 ('water', 15),
 ('corn', 15),
 ('paper', 15),
 ('pink', 15),
 ('play', 15),
 ('pull'

In [22]:
video_df2

[Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02123.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02124.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02125.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02126.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02128.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02129.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02130.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a lot/02131.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a/00295.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a/01610.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a/01611.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a/01612.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a/01613.mp4'),
 Row(path='file:/home/kristian/WLASL/videos_organized/a/01614.mp4'),
 R

In [23]:
video_df3 = pd.DataFrame(video_df2, columns=['path'])

In [24]:
video_df3.head()

,path
0,file:/home/kristian/WLASL/videos_organized/a l...
1,file:/home/kristian/WLASL/videos_organized/a l...
2,file:/home/kristian/WLASL/videos_organized/a l...
3,file:/home/kristian/WLASL/videos_organized/a l...
4,file:/home/kristian/WLASL/videos_organized/a l...


In [25]:
video_df = video_df3.join(pd.DataFrame(target, columns=['target','file']))

In [26]:
video_df.head(20)

,path,target,file
0,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02123.mp4
1,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02124.mp4
2,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02125.mp4
3,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02126.mp4
4,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02128.mp4
5,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02129.mp4
6,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02130.mp4
7,file:/home/kristian/WLASL/videos_organized/a l...,a lot,02131.mp4
8,file:/home/kristian/WLASL/videos_organized/a/0...,a,00295.mp4
9,file:/home/kristian/WLASL/videos_organized/a/0...,a,01610.mp4


In [27]:
video_df.sample(20)

,path,target,file
7810,file:/home/kristian/WLASL/videos_organized/hus...,husband,28463.mp4
16730,file:/home/kristian/WLASL/videos_organized/wha...,whale,62937.mp4
6142,file:/home/kristian/WLASL/videos_organized/fin...,finish,21941.mp4
16807,file:/home/kristian/WLASL/videos_organized/whi...,white,63205.mp4
14468,file:/home/kristian/WLASL/videos_organized/sto...,stomach,54883.mp4
8116,file:/home/kristian/WLASL/videos_organized/int...,interpreter,30261.mp4
5036,file:/home/kristian/WLASL/videos_organized/dry...,dry,17930.mp4
13296,file:/home/kristian/WLASL/videos_organized/sha...,shape,50836.mp4
574,file:/home/kristian/WLASL/videos_organized/ann...,anniversary,02611.mp4
6918,file:/home/kristian/WLASL/videos_organized/gon...,gone,65834.mp4


In [28]:
#loop to get labels and filename per row
i = 0
target2 = []
for row in video_df.path:
    subdirname = os.path.basename(os.path.dirname(video_df.path[i]))
    name = os.path.basename(video_df.path[i]).split('/')[-1]
    target2.append([subdirname])
    i = i + 1

In [29]:
target2

[['a lot'],
 ['a lot'],
 ['a lot'],
 ['a lot'],
 ['a lot'],
 ['a lot'],
 ['a lot'],
 ['a lot'],
 ['a'],
 ['a'],
 ['a'],
 ['a'],
 ['a'],
 ['a'],
 ['a'],
 ['abdomen'],
 ['abdomen'],
 ['abdomen'],
 ['abdomen'],
 ['abdomen'],
 ['abdomen'],
 ['abdomen'],
 ['able'],
 ['able'],
 ['able'],
 ['able'],
 ['able'],
 ['able'],
 ['able'],
 ['able'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['about'],
 ['above'],
 ['above'],
 ['above'],
 ['above'],
 ['above'],
 ['above'],
 ['above'],
 ['accent'],
 ['accent'],
 ['accent'],
 ['accent'],
 ['accent'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accept'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['accident'],
 ['acciden

In [30]:
video_df = video_df.join(pd.DataFrame(target2, columns=['originaltarget']))

In [31]:
video_df.sample(50)

,path,target,file,originaltarget
8258,file:/home/kristian/WLASL/videos_organized/jan...,january,30919.mp4,january
12993,file:/home/kristian/WLASL/videos_organized/sch...,school,68145.mp4,school
13384,file:/home/kristian/WLASL/videos_organized/sho...,shoes,51128.mp4,shoes
12358,file:/home/kristian/WLASL/videos_organized/rel...,rely,47140.mp4,rely
5107,file:/home/kristian/WLASL/videos_organized/dvd...,dvd,18075.mp4,dvd
14366,file:/home/kristian/WLASL/videos_organized/sta...,stare,54427.mp4,stare
11955,file:/home/kristian/WLASL/videos_organized/pus...,push,45485.mp4,push
14316,file:/home/kristian/WLASL/videos_organized/sta...,staff,54258.mp4,staff
14035,file:/home/kristian/WLASL/videos_organized/som...,somewhere,53256.mp4,somewhere
6776,file:/home/kristian/WLASL/videos_organized/gir...,giraffe,24583.mp4,giraffe


In [32]:
video_df['frequency'] = video_df['target'].map(video_df['target'].value_counts())

In [33]:
video_df.sample(20)

,path,target,file,originaltarget,frequency
8809,file:/home/kristian/WLASL/videos_organized/les...,less,32883.mp4,less,8
6811,file:/home/kristian/WLASL/videos_organized/giv...,give,24639.mp4,give,15
8876,file:/home/kristian/WLASL/videos_organized/lic...,license,33067.mp4,license,6
15919,file:/home/kristian/WLASL/videos_organized/tra...,transfer,59376.mp4,transfer,11
5005,file:/home/kristian/WLASL/videos_organized/dro...,drop,17823.mp4,drop,12
453,file:/home/kristian/WLASL/videos_organized/alp...,alphabet,02151.mp4,alphabet,5
13843,file:/home/kristian/WLASL/videos_organized/smi...,smile,52616.mp4,smile,7
364,file:/home/kristian/WLASL/videos_organized/ala...,alarm,01801.mp4,alarm,8
8722,file:/home/kristian/WLASL/videos_organized/lea...,leak,32570.mp4,leak,6
15995,file:/home/kristian/WLASL/videos_organized/tru...,truck,59774.mp4,truck,11


In [34]:
video_df = video_df[video_df['frequency'] > 14]
video_df = video_df.reset_index(drop=True)


In [35]:
video_df.sample(20)

,path,target,file,originaltarget,frequency
689,file:/home/kristian/WLASL/videos_organized/hai...,hair,26174.mp4,hair,15
603,file:/home/kristian/WLASL/videos_organized/for...,forget,22963.mp4,forget,16
1123,file:/home/kristian/WLASL/videos_organized/son...,son,70335.mp4,son,15
38,file:/home/kristian/WLASL/videos_organized/all...,all,01992.mp4,all,15
764,file:/home/kristian/WLASL/videos_organized/hot...,hot,28120.mp4,hot,18
1111,file:/home/kristian/WLASL/videos_organized/son...,son,53269.mp4,son,15
982,file:/home/kristian/WLASL/videos_organized/pap...,paper,70211.mp4,paper,15
887,file:/home/kristian/WLASL/videos_organized/mee...,meet,35513.mp4,meet,15
1301,file:/home/kristian/WLASL/videos_organized/whi...,white,63214.mp4,white,17
960,file:/home/kristian/WLASL/videos_organized/ora...,orange,40128.mp4,orange,18


In [36]:
video_df.head()

,path,target,file,originaltarget,frequency
0,file:/home/kristian/WLASL/videos_organized/acc...,accident,00618.mp4,accident,17
1,file:/home/kristian/WLASL/videos_organized/acc...,accident,00623.mp4,accident,17
2,file:/home/kristian/WLASL/videos_organized/acc...,accident,00624.mp4,accident,17
3,file:/home/kristian/WLASL/videos_organized/acc...,accident,00625.mp4,accident,17
4,file:/home/kristian/WLASL/videos_organized/acc...,accident,00626.mp4,accident,17


In [37]:
video_df.tail()

,path,target,file,originaltarget,frequency
1400,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,64314.mp4,yesterday,15
1401,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,64315.mp4,yesterday,15
1402,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,66821.mp4,yesterday,15
1403,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,66822.mp4,yesterday,15
1404,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,70347.mp4,yesterday,15


In [38]:
video_df.describe()

,frequency
count,1405.000000
mean,17.145196
std,3.330809
min,15.000000
25%,15.000000
50%,16.000000
75%,17.000000
max,34.000000


In [39]:
video_df['target'].nunique()

84

In [40]:
video_df['target'].unique()

array(['accident', 'africa', 'all', 'apple', 'bed', 'before', 'bird',
       'black', 'book', 'bowling', 'candy', 'chair', 'change', 'cheat',
       'check', 'city', 'clothes', 'color', 'computer', 'convince',
       'cook', 'cool', 'corn', 'cousin', 'cow', 'dark', 'deaf', 'decide',
       'dog', 'drink', 'family', 'fine', 'finish', 'fish', 'forget',
       'full', 'give', 'go', 'graduate', 'hair', 'hat', 'headache',
       'hearing', 'help', 'hot', 'kiss', 'language', 'last', 'later',
       'like', 'man', 'many', 'meet', 'mother', 'no', 'now', 'orange',
       'paper', 'pink', 'pizza', 'play', 'pull', 'school', 'secretary',
       'shirt', 'short', 'son', 'study', 'table', 'tall', 'thanksgiving',
       'thin', 'thursday', 'time', 'trade', 'visit', 'water', 'white',
       'who', 'woman', 'wrong', 'year', 'yes', 'yesterday'], dtype=object)

In [41]:
cap = cv2.VideoCapture(video_df.path[0])
#cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # rostro
        mp_drawing.draw_landmarks(
            frame, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
            mp_drawing.DrawingSpec(color=(0, 255, 255),
                                   thickness=1,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))

        # Mano izquieda (azul)
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Mano derecha (verde)
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Postura
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [42]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=0,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [43]:
extract_keypoints_from_file(video_df.path[0]).shape

(27, 1662)

In [44]:
cap = cv2.VideoCapture(video_df.path[0])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

27


In [45]:
type(extract_keypoints_from_file(video_df.path[0]))

numpy.ndarray

In [46]:
video_df.count()

path              1405
target            1405
file              1405
originaltarget    1405
frequency         1405
dtype: int64

In [47]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return length

In [48]:
frames_from_file(video_df.path[1404])

111

In [49]:
tqdm.pandas()
video_df['frames'] = video_df['path'].progress_apply(frames_from_file)

100%|███████████████████████████████████████| 1405/1405 [00:46<00:00, 30.51it/s]


In [50]:
video_df.frames = video_df.frames.astype('int')

In [51]:
video_df['frames'].max()

212

In [129]:
keypoints_series = video_df['path'].progress_apply(extract_keypoints_from_file)

100%|████████████████████████████████████| 1405/1405 [2:56:25<00:00,  7.53s/it]


In [130]:
keypoints_series.shape

(1405,)

In [131]:
max(keypoints.shape[0] for keypoints in keypoints_series)

212

In [132]:
min(keypoints.shape[0] for keypoints in keypoints_series)


17

In [52]:
max_len = video_df['frames'].max()

In [53]:
video_df['frames'].describe()

count    1405.000000
mean       72.436299
std        28.148461
min        17.000000
25%        53.000000
50%        71.000000
75%        89.000000
max       212.000000
Name: frames, dtype: float64

In [135]:
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
for i, keypoints in enumerate(keypoints_series):
    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [1]:
keypoints_np.shape

NameError: name 'keypoints_np' is not defined

In [137]:
np.save("/home/kristian/Capstone/keypoints/mediapipe_keypoints.npy", keypoints_np)

In [1]:
import numpy as np
keypoints_np=np.load('/home/kristian/Capstone/keypoints/mediapipe_keypoints.npy')

In [2]:
keypoints_np.shape

(1405, 212, 1662)

# Preprocessing

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import to_categorical
import numpy as np

2023-08-03 12:45:49.265854: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-03 12:45:50.556648: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
video_df.head()

NameError: name 'video_df' is not defined

In [61]:
encoder = OrdinalEncoder()

In [62]:
video_df['cat_label'] = encoder.fit_transform(video_df[['target']]).astype(int)

In [63]:
video_df.head()

,path,target,file,originaltarget,frequency,frames,cat_label
0,file:/home/kristian/WLASL/videos_organized/acc...,accident,00618.mp4,accident,17,27,0
1,file:/home/kristian/WLASL/videos_organized/acc...,accident,00623.mp4,accident,17,104,0
2,file:/home/kristian/WLASL/videos_organized/acc...,accident,00624.mp4,accident,17,109,0
3,file:/home/kristian/WLASL/videos_organized/acc...,accident,00625.mp4,accident,17,34,0
4,file:/home/kristian/WLASL/videos_organized/acc...,accident,00626.mp4,accident,17,44,0


In [64]:
video_df.tail()

,path,target,file,originaltarget,frequency,frames,cat_label
1400,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,64314.mp4,yesterday,15,64,83
1401,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,64315.mp4,yesterday,15,109,83
1402,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,66821.mp4,yesterday,15,64,83
1403,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,66822.mp4,yesterday,15,62,83
1404,file:/home/kristian/WLASL/videos_organized/yes...,yesterday,70347.mp4,yesterday,15,111,83


In [65]:
label_map = {label:num for num, label in enumerate(video_df['target'].unique())}

In [66]:
label_map

{'accident': 0,
 'africa': 1,
 'all': 2,
 'apple': 3,
 'bed': 4,
 'before': 5,
 'bird': 6,
 'black': 7,
 'book': 8,
 'bowling': 9,
 'candy': 10,
 'chair': 11,
 'change': 12,
 'cheat': 13,
 'check': 14,
 'city': 15,
 'clothes': 16,
 'color': 17,
 'computer': 18,
 'convince': 19,
 'cook': 20,
 'cool': 21,
 'corn': 22,
 'cousin': 23,
 'cow': 24,
 'dark': 25,
 'deaf': 26,
 'decide': 27,
 'dog': 28,
 'drink': 29,
 'family': 30,
 'fine': 31,
 'finish': 32,
 'fish': 33,
 'forget': 34,
 'full': 35,
 'give': 36,
 'go': 37,
 'graduate': 38,
 'hair': 39,
 'hat': 40,
 'headache': 41,
 'hearing': 42,
 'help': 43,
 'hot': 44,
 'kiss': 45,
 'language': 46,
 'last': 47,
 'later': 48,
 'like': 49,
 'man': 50,
 'many': 51,
 'meet': 52,
 'mother': 53,
 'no': 54,
 'now': 55,
 'orange': 56,
 'paper': 57,
 'pink': 58,
 'pizza': 59,
 'play': 60,
 'pull': 61,
 'school': 62,
 'secretary': 63,
 'shirt': 64,
 'short': 65,
 'son': 66,
 'study': 67,
 'table': 68,
 'tall': 69,
 'thanksgiving': 70,
 'thin': 71,
 'th

In [67]:
video_df.sample(10)

,path,target,file,originaltarget,frequency,frames,cat_label
685,file:/home/kristian/WLASL/videos_organized/hai...,hair,26169.mp4,hair,15,136,39
458,file:/home/kristian/WLASL/videos_organized/dar...,dark,69290.mp4,dark,15,69,25
356,file:/home/kristian/WLASL/videos_organized/con...,convince,13141.mp4,convince,15,17,19
629,file:/home/kristian/WLASL/videos_organized/giv...,give,24639.mp4,give,15,23,36
137,file:/home/kristian/WLASL/videos_organized/boo...,book,07069.mp4,book,34,30,8
893,file:/home/kristian/WLASL/videos_organized/mee...,meet,35521.mp4,meet,15,61,52
535,file:/home/kristian/WLASL/videos_organized/fam...,family,20981.mp4,family,16,78,30
322,file:/home/kristian/WLASL/videos_organized/col...,color,11778.mp4,color,16,85,17
47,file:/home/kristian/WLASL/videos_organized/all...,all,69206.mp4,all,15,89,2
905,file:/home/kristian/WLASL/videos_organized/mot...,mother,36937.mp4,mother,18,46,53


In [4]:
Y = encoder.fit_transform(video_df[['target']])

NameError: name 'encoder' is not defined

In [69]:
Y

array([[ 0.],
       [ 0.],
       [ 0.],
       ...,
       [83.],
       [83.],
       [83.]])

In [70]:
n_classes = 84
print("Shape before one-hot encoding: ", Y.shape)
Y = to_categorical(Y, n_classes)
print("Shape after one-hot encoding: ", Y.shape)

Shape before one-hot encoding:  (1405, 1)
Shape after one-hot encoding:  (1405, 84)


In [4]:
X = keypoints_np

In [5]:
X.shape

(1405, 212, 1662)

In [6]:
#np.save("/home/kristian/Capstone/keypoints/labels_array.npy", Y)

In [7]:
Y=np.load('/home/kristian/Capstone/keypoints/labels_array.npy')

In [8]:
type(Y)

numpy.ndarray

In [9]:
Y.shape

(1405, 84)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)

# Model

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

In [21]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [22]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
model = Sequential()


In [24]:
model.add(LSTM(128, return_sequences=True, activation='tanh', input_shape=(212,1662)))
model.add(LSTM(64, return_sequences=True, activation='tanh'))
model.add(LSTM(128, return_sequences=False, activation='tanh'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(Y.shape[1], activation='softmax'))

In [25]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_7 (LSTM)               (None, 212, 128)          916992    
                                                                 
 lstm_8 (LSTM)               (None, 212, 64)           49408     
                                                                 
 lstm_9 (LSTM)               (None, 128)               98816     
                                                                 
 dense_6 (Dense)             (None, 128)               16512     
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dense_8 (Dense)             (None, 84)                5460      
                                                                 
Total params: 1095444 (4.18 MB)
Trainable params: 1095

In [27]:
optimizer = Adam(learning_rate=1)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [28]:
model.fit(X_train, Y_train, epochs=20, callbacks=[tb_callback],batch_size =16)

2023-08-03 12:47:39.376167: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1584138624 exceeds 10% of free system memory.
2023-08-03 12:47:41.093919: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1584138624 exceeds 10% of free system memory.


Epoch 1/20


2023-08-03 12:47:48.590685: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-03 12:47:48.706007: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fe5bc014650 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-03 12:47:48.706046: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-03 12:47:48.711802: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-03 12:47:48.838794: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-08-03 12:47:48.920033: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


71/71 [==============================] - 13s 67ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 2/20
71/71 [==============================] - 5s 67ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 3/20
71/71 [==============================] - 5s 67ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 4/20
71/71 [==============================] - 5s 69ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 5/20
71/71 [==============================] - 5s 72ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 6/20
71/71 [==============================] - 5s 74ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 7/20
71/71 [==============================] - 6s 92ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 8/20
71/71 [==============================] - 6s 78ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 9/20
71/71 [==============================] - 7s 92ms/step - loss: nan - categorical_accuracy: 0.0125
Epoch 10/20
71/71 [===================

In [30]:
X.shape

(1405, 212, 1662)

In [29]:
X

array([[[ 0.49979177,  0.2442503 , -0.93340957, ...,  0.33342743,
          0.61645138, -0.02810866],
        [ 0.49981225,  0.24427561, -0.93794864, ...,  0.3676649 ,
          0.59613639, -0.02663341],
        [ 0.49991232,  0.24416235, -0.93785214, ...,  0.37068117,
          0.59557921, -0.02488603],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.50543946,  0.25723302, -0.9598015 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.50475228,  0.2565645 , -0.9301303 , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.50425142,  0.25583747, -0.92295343, ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.        ,  0.        ,  0.        , ...,  